<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# How do I use Riva ASR APIs with out-of-the-box models?

This tutorial walks you through the basics of Riva Speech Skills ASR Services, specifically covering how to use Riva ASR APIs with out-of-the-box models.

## NVIDIA Riva Overview

NVIDIA Riva is a GPU-accelerated SDK for building Speech AI applications that are customized for your use case and deliver real-time performance. <br/>
Riva offers a rich set of speech services such as:

- Automated speech recognition (ASR)
- Text-to-Speech synthesis (TTS)

In this tutorial, we will interact with the automated speech recognition (ASR) APIs.

For more information about Riva, refer to the [Riva developer documentation](https://developer.nvidia.com/riva).

## Transcription with Riva ASR APIs

ASR takes an audio stream or audio buffer as input and returns one or more text transcripts, along with additional optional metadata. Speech recognition in Riva is a GPU-accelerated compute pipeline, with optimized performance and accuracy.  
Riva provides state of the art OOTB(out-of-the-box) models and pipelines for multiple languages, like English, Spanish, German, Russian and Mandarin, that can be easily deployed with the Riva Quick Start Scripts. Riva also supports easy customization of the ASR pipeline, in various ways, to meet your specific needs.  
Refer to the [Riva ASR documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/asr/asr-overview.html) for more information.  

Now, let's generate the transcripts using Riva APIs, for some sample audio clips, with an OOTB pipeline, starting with English.

#### Import the Riva client libraries

Let's import some of the required libraries, including the Riva Client libraries.

In [ ]:
import io
import IPython.display as ipd
import grpc

import riva.client

#### Create a Riva client and connect to the Riva Speech API server

The following URI assumes a local deployment of the Riva Speech API server is on the default port. In case the server deployment is on a different host or via a Helm chart on Kubernetes, use an appropriate URI.

In [ ]:
auth = riva.client.Auth(uri='localhost:50051')

riva_asr = riva.client.ASRService(auth)

### Offline recognition for English

You can use Riva ASR in either streaming mode or offline mode. In streaming mode, a continuous stream of audio is captured and recognized, producing a stream of transcribed text. In offline mode, an audio clip of a set length is transcribed to text. <br> 
Let's look at an example showing offline ASR API usage for English:

#### Make a gRPC request to the Riva Speech API server
Riva ASR API supports `.wav` files in pulse-code modulation (PCM) format; including `.alaw`, `.mulaw`, and `.flac` formats with single channel. 

Now, let's make a gRPC request to the Riva Speech server for ASR with a sample `.wav` file in offline mode. Start by loading the audio.

In [ ]:
# This example uses a .wav file with LINEAR_PCM encoding.
# read in an audio file from local disk
path = "audio_samples/en-US_sample.wav"
with io.open(path, 'rb') as fh:
    content = fh.read()
ipd.Audio(path)

Next, create an audio `RecognizeRequest` object, setting the configuration parameters as required.

In [ ]:
# Set up an offline/batch recognition request
config = riva.client.RecognitionConfig()
#config.encoding = ra.AudioEncoding.LINEAR_PCM    # Audio encoding can be detected from wav
#config.sample_rate_hertz = 0                     # Sample rate can be detected from wav and resampled if needed
config.language_code = "en-US"                    # Language code of the audio clip
config.max_alternatives = 1                       # How many top-N hypotheses to return
config.enable_automatic_punctuation = True        # Add punctuation when end of VAD detected
config.audio_channel_count = 1                    # Mono channel

Finally, submit the request to the server.

In [ ]:
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript:", asr_best_transcript)

print("\n\nFull Response Message:")
print(response)

#### Understanding ASR API parameters

Riva ASR supports a number of options while making a transcription request to the gRPC endpoint, as shown in the previous section. Let's learn more about these parameters:
- `encoding` - Type of audio encoding of the input audio file. Supports (`LINEAR_PCM`, `FLAC`, `MULAW` or `ALAW`). Can be detected from audio file
- `sample_rate_hertz` - Sample rate can be detected from the audio `.wav` file and resampled if needed.
- `language_code` - Language of the input audio. "en-US" represents English (US). Other options include (`es-US`, `de-DE`, `ru-RU`, `zh-CN`). We will explore ASR for non-English languages in the next section.
- `enable_automatic_punctuation` - Adds a punctuation at the end of VAD (Voice Activity Detection).
- `audio_channel_count` - Number of audio channels. Typical microphones have 1 audio channel.

With this, we come to an end for the introduction to Riva's offline python client for ASR. Feel free to read more about the Riva's ASR proto [here](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/reference/protos/protos.html#protobuf-docs-asr).